A full report consisting of all of the following components (15 marks):<br /><br />
Introduction where you discuss the business problem and who would be interested in this project.<br /><br />
Data where you describe the data that will be used to solve the problem and the source of the data.<br /><br />
Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, and what machine learnings were used and why.<br /><br />
Results section where you discuss the results.<br /><br />
Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.<br /><br />
Conclusion section where you conclude the report.<br /><br />

## Coursera - IBM Data Science Capstone Project ##

### Brian Lee ###
BrianTrewLee@gmail.com<br />
LinkedIn: https://www.linkedin.com/in/brian-lee-data




## 1. Introduction ##

If I were to open a Starbucks, where should I locate it? Clearly, the location is one of the most important business decisions for this venture. Is there a science to locating a StarBucks? Can we create a machine learning model to predict where a StarBucks is likely to be located? I'd like to find out.

If a reliable model can be made, it could be used in the process of opening a store. It could be used as a final sanity check, or at the beginning stage, to select a promising location. If a trustworthy model predicts with a high degree of confidence that a Starbucks should be located in an area, but there is not one there yet, perhaps that is an opportunity.

Any person interested in opening a Starbucks should be interested in these results. This includes myself, other potential franchisees, and the Starbucks corporation itself, which operates many of it's own stores. The Starbucks competition might also be interested as they could possibly gain competitive insights. I also believe others might be interested in this procedure, as it might be applied to predicting the location of other entities, based on the same sort of data.

The main purpose of this project is to prove the concept of predicting Starbuck's locations in general. I suspect it may work better or worse depending on the locations chosen for training and prediction, the radius size, and the specific features used in the model. I may vary those factors in order to prove the concept, which could then be applied carefully to a particular geographic area of interest at a later date.

### 2. Data ###

Foursquare (https://foursquare.com) is a company that crowdsources location data, tying latitude-longitude coordinates (and other things) with public venues, including many businesses, such as Starbucks locations. The features for the Starbucks location classifier will come from venue data from the Fourquare API. I will use venue category names (such as 'bar', 'Chinese restaurant', 'coffee shop') as features to classify an area as either an area with a Starbucks, or an area without one. I will use the venue name (e.g. 'Starbucks') to determine whether or not the venue is a Starbucks. An area containing a Starbucks contains at least one venue named 'Starbucks' within the radius supplied to the Foursquare API's explore' endpoint.

An area will be a circular area with a given radius. I used a radius of 300m for this project. Within that radius, the Foursquare API will return venues. The larger the radius, the more venues that could be anticipated, and the higher likelihood of a Starbucks, but a larger radius is also less useful for business use.

The quality of the Foursquare venue data is good, but not perfect. I noticed venues returned as "SUBWAY" separately from other venues returned as "Subway." I cleaned that by altering the case of the venue names after received by the API. I also noticed that the crowdsourced latitude-longitude coordinates for Starbucks locations often include duplicative results, slightly offset. For instance, there could be a cluster of tightly packed coordinates returned as Starbucks locations, giving the impression of many Starbucks next to each other. I suspect this actually reflects different geographic coordinates representing a single Starbucks venue. Additionally, there are areas (for instance in my validation area in Illinois) where no venues are reported within the radius. In some cases, it can be seen that some venues (such as churchs) are in fact located nearby.

The areas chosen for training/testing and validation data contain a mix of areas with Starbuck's and those without. I selected a geographic rectangle in San Francisco for training/testing. With a reasonably small radius, most points will have true labels of 0 (or not near a Starbucks.) In order to balance the classes for better binary classification performance, I added a large number of additional geographic points to the training set. These additional coordinates come from the Kaggle dataset (https://www.kaggle.com/starbucks/store-locations) containing the latitude and longitude coordinates of 25,600 worldwide Starbucks locations. The idea was that these added points should have true labels of 1 (near a Starbucks), and most do. However, I discovered that they do not all. To that end, the best balancing would be a ratio of about 1.5/1. However, I found that balancing 1:1 seemed sufficient for training performance. The fact that these added points don't all appear to have labels of 1 is distressing. Perhaps that is a sign that something is wrong, or perhaps it is due to different data sources not agreeing on exactly where a current Starbucks venue is located. In any event, my training data is comprised of a grid of points inside a rectangle, combined with an equal number of points throughout California. 

I wanted to validate the results on a completely separate area, with a naturally occurring distribution of classes. I chose a rectangular area in Illinois, near Chicago, with a decent number of starbucks locations according to Google Maps. When I got the venue category names I used to create onehot features, I discovered that I had a slightly different set of features. There were a few (3) venue category types that did not occur in my training/testing set, and a few dozen (~30) that occurred in the validation data that did not occur in the training/testing data. I could have retrained my model with a superset of features that wold satisfy both regimes, but I chose instead to reformat my validation features to fit my trained model. I added three dummy columns to the validation data, and threw out 30 columns, resulting in a feature set of 452. I felt this approach was a good approach to test the generalizability and durability of the trained model.





I have done some exploratory work in a particular area of San Francisco. I located Starbucks with Google Maps and got latitude and longitude points making a box around that area. I could choose the train and test points at random within an area, or I could generate a grid covering an entire area. Given the constraints my Foursquare Developer account places on my searches, and knowing that binary classifiers work better with a good number of examples of both classes, I may want to generate areas that I know will be heavy with Starbucks locations. To that end, I discovered a  I may or may not use that resource.

### 3. Methodology ###

I will generate a corpus of geographic points, and using a reasonable radius (perhaps 300m) a corpus of geographic areas. Each geographic area will have a feature set based on json data returned by the Foursquare API's 'explore' endpoint. I plan to use the category names of the returned venues as features for that area. I will use the venue names to determine whether oir not a Starbucks is in that area, and generate the labels from that. 

Once I have labelled data, I will use sklearn.model_selection.train_test_split to generate tarining and testing sets. Then I  can use any of a host of sklearn classifiers to generate predictive models. I have done some exploratory work with a random forest.

In [ ]:
# @hidden_cell
CLIENT_ID='0C4GFTHUCEID10D2EF5OUUVTPDE3LRKIKOCWYOXSVTN0L3FB'
CLIENT_SECRET='UC52VVB1WLAD40QJIBUYHX54AMWFXPGJEMM4C0OZRGCOHWFZ'
VERSION='20180608'
LIMIT = 100

In [ ]:
# import libraries

import urllib.request # open and read URLs

import json # handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests # handle requests
import pandas as pd # process data as dataframes with Pandas
import numpy as np # handle data in a vectorized manner with NumPy

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't installed the GeoPy geocoding library yet
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

##!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't installed the Folium library yet
import folium # map rendering library

# Matplolib plotting library and associated modules
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
import matplotlib.colors as colors

import random
import geopy
from geopy.distance import VincentyDistance
from sklearn.model_selection import train_test_split
import sklearn as sk  
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.tree import export_graphviz
from subprocess import call
from IPython.display import Image
from sklearn.feature_selection import RFECV
from sklearn import svm
from sklearn.cluster import KMeans # for K-Means clustering with Scikit-Learn

print("Libraries imported!")

In [ ]:
eliminate_starbucks_venue_rows = True # It seems like we should eliminate actual Starbucks locations from our training data
verbose = False

In [ ]:
# Define a function to gather recommended venues, with specifically the name and category, using the explore API
def getNearbyVenues(latitudes, longitudes, radius=300, limit=100):
    
    count = 0
    venues_list = []
    test_list = []
    for lat, lng in zip(latitudes, longitudes):
        #create a unique name for this point that is easier to deal with than a combination of lat and lon
        point_name = "point"+str(count).zfill(5)
        if verbose:
            print(point_name,':',lat,'-',lng)
         
        else:
            # progress indicator for the impatient among us
            if(count%100 == 0):
                print("\r{0}".format(round((float(count)/len(latitudes))*100),2)+"% done")
            
        # create the API request URL
        url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            point_name,
            lat, 
            lng, 
            v["venue"]["name"], 
            v["venue"]["location"]["lat"], 
            v["venue"]["location"]["lng"],  
            v["venue"]["location"]["distance"], 
            v["venue"]["categories"][0]["name"]) for v in results])        

   
        if (len(results)<1):
            venues_list.append([(
                point_name,
                lat, 
                lng, 
                "", 
                0, 
                0,  
                0, 
                "No venues within radius")])
                
                
        test_list.append([(point_name) for v in results])
        
    
        count = count + 1
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    print("nearby_venues length:" + str(len(nearby_venues)))      
    
    nearby_venues.columns = ["Point Name",
                  "Neighborhood Latitude", 
                  "Neighborhood Longitude", 
                  "Venue", 
                  "Venue Latitude", 
                  "Venue Longitude", 
                  "Venue Distance",           
                  "Venue Category"]
    print("# of unique points in nearby_venues ="+str(len(nearby_venues["Point Name"].unique())))
    return(nearby_venues)

In [ ]:
# It is possible that Foursqaure may have incorrect data
# I have seen areas with venues where the API returned no results
def getStarbucksLocations(lat, lng, limit=200):
    
    count = 0
    starbucks_list = []
    #for lat, lng in zip(latitudes, longitudes):
        #create a unique name for this point that is easier to deal with than a combination of lat and lon
    this_radius = 300
         
    
            
        # create the API request URL
       # url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        #url = "https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}".format(
    #url = "https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&limit={}&ne={},{}&sw={},{}&query={}".format(
    url = "https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&limit={}&radius={}&ll={},{}&query={}".format(
          
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            limit,
            this_radius,
            lat,
            lng,
            "Starbucks")
    sb_coords = []
    # make the GET request
   # results = requests.get(url).json()["response"]["venues"]["location"]
    results = requests.get(url).json()["response"]["venues"]
    for result in results:
        sb_coords.append((result["location"]["lat"],result["location"]["lng"]))
        
    if (len(results)<1):
        if verbose ==True:
            print("no starbucks location results")
    return(sb_coords)


In [ ]:
sf_lat=37.7749
sf_lon=-122.4194

In [ ]:
llat=37.7
ulat = 37.807957
llon = -122.494329
ulon = -122.395276

points = []
val_points = []
radius = 300 #in meters
current_lat = ulat
current_lon = llon

d = radius/1000 #in kilometers
d=d*2
b=90 #in degrees 90 = east,180=south,270=west
# given: lat1, lon1, b = bearing in degrees, d = distance in kilometers      
 
def lon_pass(current_lat, current_lon, ulat, ulon, pnts):
    while (current_lat <= ulat) and (current_lon <= ulon) and (current_lat >= llat):    
        origin = geopy.Point(current_lat, current_lon)
        destination = VincentyDistance(kilometers=d).destination(origin, b)
        current_lat, current_lon = destination.latitude, destination.longitude
        if verbose==True:
            print(current_lat, current_lon)
        pnts.append(list((current_lat, current_lon)))
    return(pnts)
        
 
        

def generate_points(llat, llon, ulat, ulon):
    pnts=[]
    current_lat = ulat
    current_lon = llon   
    #while (current_lat <= ulat) and (current_lon <= ulon) and (current_lat >= llat) and (current_lon >= llon):
    while (current_lat <= ulat) and (current_lon <= ulon) and (current_lat >= llat):
        pnts = lon_pass(current_lat, current_lon, ulat, ulon, pnts)
        print('new row of coordinates')  
        current_lon=llon#move back to beginning of row   
        origin = geopy.Point(current_lat, current_lon)
        destination = VincentyDistance(kilometers=d).destination(origin, 180)#move south one step for next row
        current_lat, current_lon = destination.latitude, destination.longitude       
    return(pnts)

In [ ]:
points = generate_points(llat, llon, ulat, ulon)


#### We many need more examples of the near-Starbucks class for a balanced set - so get a bunch of points we know are near to Starbucks
I pulled coordinates from the Kaggle Starbucks locations dataset<br />
The precision of the Kaggle set wasn't good enough to ensure the Starbucks location would be within the radius chosen, so I found a different dataset with higher precision

In [ ]:
#starbucks_heavy_coordinates = pd.read_csv('data/directory-us-ca.csv') #this file gave lat-lon tothe hundreths, but that isn't good enough to ensure a Starbucks is within my radius
starbucks_heavy_coordinates = pd.read_csv('data/starbucks_us_ca_locations.csv',header=None)
starbucks_heavy_coordinates.columns = ["Longitude", "Latitude", "Location", "Address"]
# These coordinates aren't always precise enough to trigger a Starbucks location in the Foursquare API within the required radius
# For that reason, pulling an equal number of coordinate points from this set doesn't necessarily balance the classes exactly
# Sometimes two different coordinate points in this set are given for a single Starbucks location. They are rare enough to be ignored for now.

In [ ]:
starbucks_heavy_coordinates.head()

In [ ]:
#num_needed_to_balance = int(len(points)*1.5) #This balances the classes better
num_needed_to_balance = len(points) #This does a good enough job of balancing the classes
num_needed_to_balance

In [ ]:
shc_lat, shc_lon = starbucks_heavy_coordinates.Latitude, starbucks_heavy_coordinates.Longitude

nnl=(shc_lat[0:num_needed_to_balance],shc_lon[0:num_needed_to_balance])
pnnl = pd.DataFrame(nnl)
pnnl.shape
pnnl = pnnl.transpose()
pnnl
newlats=pnnl['Latitude']
newlons=pnnl['Longitude']
newlats
points = pd.DataFrame(points)
current_end = len(points)
for i in range(0,num_needed_to_balance):
    lenp = len(points)
    points = points.set_value(lenp, 0, newlats[i])
    points = points.set_value(lenp, 1, newlons[i])  

In [ ]:
points.shape

shuffle the points

In [ ]:
points = points.sample(frac=1).reset_index(drop=True)
points.head()

In [ ]:
points.tail()

In [ ]:
map_training_data = folium.Map(location=[ulat, ulon], zoom_start=11)


# add markers to map
for lat, lng in zip(points[0],points[1]):
    if (lat <= ulat) and (lng*1.00005 <= ulon) and (lat >= llat) and (lng >=llon):#had to multiply by a tad to avoid rounding errors
        marker_color = 'navy'
    else:
        marker_color = 'brown'
        
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=marker_color,
        #fill=True,
        fill=False,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_training_data)  
    

In [ ]:

map_training_data_legend_html =   '''
                <div style="position: fixed; 
                            bottom: 50px; left: 44px; width: 148px; height: 130px; 
                            border:2px solid grey; z-index:9999; font-size:14px;
                            #background: white;
                            ">
                              <div style="font-size:10px;"><br />&nbsp; Training classes balanced<br /> 
                              &nbsp; with known Starbucks<br />
                              &nbsp; locations throughout<br />
                              &nbsp; California<br />  
                              </div>
                              
                              &nbsp;&nbsp;&nbsp; unknown: &nbsp; <i class="fa fa-circle-o style="color:navy"></i><br>
                              &nbsp;&nbsp;&nbsp; known: &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <i class="fa fa-circle-o" style="color:brown">
                              </i>
                </div>
                ''' 
map_training_data.get_root().html.add_child(folium.Element(map_training_data_legend_html))

In [ ]:
map_training_data

In [ ]:
venues = getNearbyVenues(latitudes = points[0],
                              longitudes =  points[1]
                               )


In [ ]:
venues.tail()

In [ ]:
venues['Venue'].unique()
venues['Venue'].value_counts()

In [ ]:
venues['Venue Category'].unique()

In [ ]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['point_name'] = venues['Point Name'] 
onehot['point_lat'] = venues['Neighborhood Latitude'] 
onehot['point_lon'] = venues['Neighborhood Longitude'] 
onehot['isStarbucks'] = (venues['Venue'] == "Starbucks")
# move neighborhood column to the first column
fixed_columns = [onehot.columns[-3]] + [onehot.columns[-2]] + [onehot.columns[-1]] + list(onehot.columns[:-3])
onehot = onehot[fixed_columns]

onehot.isStarbucks = onehot.isStarbucks.astype(int)

onehot.head()
onehot['isStarbucks'].value_counts()

In [ ]:
venues.head()

In [ ]:
len(onehot)

In [ ]:
points_near_starbucks = onehot[onehot.isStarbucks > 0]["point_name"]
if eliminate_starbucks_venue_rows == True:
    #keep the isStarbucks flag but throw out other venue info
    ##points_near_starbucks = onehot[onehot.isStarbucks > 0]["point_name"]
    indexes_to_remember = onehot[onehot.isStarbucks > 0].index
    points_near_starbucks

In [ ]:
points_near_starbucks_df = pd.DataFrame(points_near_starbucks)
points_near_starbucks_df.head()

for point_name in points_near_starbucks_df.point_name:
    onehot.loc[onehot.point_name == point_name,"isStarbucks"] = 1  

In [ ]:
len(onehot)

In [ ]:
if eliminate_starbucks_venue_rows == True:
    # eliminate venues called Starbucks from the training data, as that unfairly 
    # improves prediction, even when de-indentified
    # For example, the presence of a coffe shop at those coordinates
    # makes for a powerfully predictive training feature
    for ind in indexes_to_remember:
        onehot = onehot.drop([ind], axis=0)

In [ ]:
#shorter now if we removed Starbucks venues
len(onehot)

In [ ]:
len(onehot.columns)

My intuition is that an area with more venues is more business-centric, <br />
and that would affect the likelihood of a Starbucks being located nearby<br />
Let's create a feature for the number of venues nearby

In [ ]:
onehot['number_of_venues_within_radius'] = 1

In [ ]:
len(onehot.columns)

In [ ]:
# create the new feature 'number_of_venues_within_radius'
# have to combine means and counts
grouper = onehot.groupby('point_name').mean().reset_index()
counter = onehot.groupby('point_name').count().reset_index()
grouped = grouper
grouped.iloc[:,-1] = counter.iloc[:,-1]
max_of_venue_count = grouped.iloc[:,-1].max()
# normalize the new feature
grouped.iloc[:,-1] = grouped.iloc[:,-1]/max_of_venue_count
grouped.iloc[:,-1].describe()

In [ ]:
verbose = False
num_top_venues = 5
count = 0
for point in grouped['point_name']:
    temp = grouped[grouped['point_name'] == point].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[2:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    if verbose == True:
        print("----"+point+"----")
        print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
        print('\n')
    else:
        if count % 100 == 0:
            print("----"+point+"----")
            print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
            print('\n')
    count = count + 1

In [ ]:
grouped.head()

In [ ]:
y = grouped['isStarbucks'] > 0
y = pd.DataFrame(y)
y = y.isStarbucks.astype(int)
print(y.value_counts())
X = grouped
print(X.shape)
X.drop(columns=['point_name', 'point_lat', 'point_lon', 'isStarbucks'], inplace=True)


In [ ]:
X.columns

Create a random feature<br />
Let's see how many of our features do no better than a randomly generated feature

In [ ]:
X['random'] = np.random.random(size=len(X))

In [ ]:
print(X.columns)
print(X.shape)

In [ ]:
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
pdytrain=pd.DataFrame(y_train)
pdytest=pd.DataFrame(y_test)
pdytrain['isStarbucks'].value_counts()

In [ ]:
pdytest['isStarbucks'].value_counts()

In [ ]:
class CLF:
    def __init__(self, name):
        self.name = name

    def fit(self):
        pass

In [ ]:
class SVM:
    def __init__(self, name, kernel_type, X_train, y_train, X_test, y_test, X_val, y_val):
        self.ClassifierType = "Support Vector Machine"
        self.name = name 
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.X_val = X_val
        self.y_val = y_val        
        self.kernel_type = kernel_type
        self.svm_model = svm.SVC(kernel=kernel_type, C=1, gamma=1, probability=True) 
        print(self.ClassifierType + " - " + name + " created")
    def fit(self):
        fit = self.svm_model.fit(X_train, y_train)
    def predict(self, X):
        predicted = self.svm_model.predict(X)
    def score(self, X, y):
        score = self.svm_model.score(X, y)
    def print_scores(self):
        self.svm_model.score(X_train,y_train)
        print(self.name + " Train accuracy = " + str(self.svm_model.score(X_train,y_train)))
        print(self.name + " Test accuracy = " + str(self.svm_model.score(X_test,y_test)))
        print(self.name + " Validation accuracy = " + str(self.svm_model.score(X_val,y_val)))        
        svm_prob_y_train = self.svm_model.predict_proba(X_train)
        svm_prob_y_test = self.svm_model.predict_proba(X_test)      
        svm_prob_y_val = self.svm_model.predict_proba(X_val)              
        print(self.name + " Train ROC AUC = " + str(roc_auc_score(y_train, svm_prob_y_train[:,1])))
        print(self.name + " Test ROC AUC = " + str(roc_auc_score(y_test, svm_prob_y_test[:,1]))) 
        print(self.name + " Validation ROC AUC = " + str(roc_auc_score(y_val, svm_prob_y_val[:,1])))         

In [ ]:
class RF:
    rfc = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=0)
    def __init__(self, name, X_train, y_train, X_test, y_test, X_val, y_val):
        self.ClassifierType = "Random Forest"
        self.name = name 
        self.X_train = X_train
        self.y_train = y_train
        self.y_train = X_test
        self.y_train = y_test
        self.y_train = X_val
        self.y_train = y_val
        
        self.rfc = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=0)
        print(self.ClassifierType + " - " + name + " created")
        
    #def visualize_first_estimator(self):
        
    def get_estimator(self, estimator_num):        
        estimator =  self.f.estimators_[estimator_num]
        return(estimator)
        
    def fit(self):
        self.f = self.rfc.fit(X_train,y_train)
    def predict(self, X):
        self.predictions = RF.rfc.predict(X)   
        #print(self.predictions)
    def predict_all(self, X):
        self.test_predictions = RF.rfc.predict(X_test)   
        self.val_predictions = RF.rfc.predict(X_val)           
        #print(self.predictions)              
        
    def get_and_print_results(self):
        self.prob_y_train = self.rfc.predict_proba(X_train)
        self.prob_y_test = self.rfc.predict_proba(X_test)
        if len(X_val) > 0 and len(y_val) > 0 :
            self.prob_y_val = self.rfc.predict_proba(X_val)
        
        self.num_features = self.f.n_features_
        print(self.num_features)        
        self.train_accuracy = self.rfc.score(X_train,y_train)
        self.test_accuracy = self.rfc.score(X_test,y_test)
        if len(X_val) > 0 and len(y_val) > 0 :
            self.validation_accuracy = self.rfc.score(X_val,y_val)        
        
        self.feature_importances = self.rfc.feature_importances_
        print(self.name + " Train accuracy = " + str(self.train_accuracy))  
        print(self.name + " Train ROC AUC = " + str(roc_auc_score(y_train, self.prob_y_train[:,1])))
        print(self.name + " Test accuracy = " + str(self.test_accuracy))  
        print(self.name + " Test ROC AUC = " + str(roc_auc_score(y_test, self.prob_y_test[:,1])))
        
        print(self.name + " Validation accuracy = " + str(self.validation_accuracy))  
        print(self.name + " Validation ROC AUC = " + str(roc_auc_score(y_val, self.prob_y_val[:,1])))
        print(self.name + " Num Features: " + str(self.num_features))  
       
        self.feature_importance_info = []
        for i in range(0,self.num_features):
            if self.feature_importances[i] >= 0:
                self.feature_importance_info.append((self.feature_importances[i],X_train.columns[i]))
    
        self.feature_importance_df = pd.DataFrame(self.feature_importance_info)
        self.feature_importance_df.rename(columns={0: "Feature Importance", 1: "Feature"}, inplace=True)
        self.feature_importance_df.sort_values(by="Feature Importance",ascending=False,inplace=True)    
        
        self.feature_importance_df.set_index('Feature',inplace=True)
        self.feature_importance_df       

        
        
    def get_and_print_results_without_validation(self):
        self.prob_y_train = self.rfc.predict_proba(X_train)
        self.prob_y_test = self.rfc.predict_proba(X_test)
        #if len X_val >0 and len y_val > 0 :
        #    self.prob_y_val = self.rfc.predict_proba(X_val)
        
        self.num_features = self.f.n_features_
        print(self.num_features)        
        self.train_accuracy = self.rfc.score(X_train,y_train)
        self.test_accuracy = self.rfc.score(X_test,y_test)
        
        self.feature_importances = self.rfc.feature_importances_
        print(self.name + " Train accuracy = " + str(self.train_accuracy))  
        print(self.name + " Train ROC AUC = " + str(roc_auc_score(y_train, self.prob_y_train[:,1])))
        print(self.name + " Test accuracy = " + str(self.test_accuracy))  
        print(self.name + " Test ROC AUC = " + str(roc_auc_score(y_test, self.prob_y_test[:,1])))
        
        print(self.name + " Validation accuracy = " + str(self.validation_accuracy))  
        print(self.name + " Validation ROC AUC = " + str(roc_auc_score(y_val, self.prob_y_val[:,1])))
        
        
        print(self.name + " Num Features: " + str(self.num_features))  
       
        self.feature_importance_info = []
        for i in range(0,self.num_features):
            if self.feature_importances[i] >= 0:
                self.feature_importance_info.append((self.feature_importances[i],X_train.columns[i]))
    
        self.feature_importance_df = pd.DataFrame(self.feature_importance_info)
        self.feature_importance_df.rename(columns={0: "Feature Importance", 1: "Feature"}, inplace=True)
        self.feature_importance_df.sort_values(by="Feature Importance",ascending=False,inplace=True)    
        
        self.feature_importance_df.set_index('Feature',inplace=True)
        self.feature_importance_df       
        
        
        
print("refreshed")        

In [ ]:
##### validate #####

In [ ]:
#starbucks heavy area in Illinois
#41.916715, -87.989170
#41.843114, -87.779305
validate_llat = 41.843114
validate_llon = -87.989170
validate_ulat = 41.916715
validate_ulon = -87.779305
val_points = generate_points(validate_llat, validate_llon, validate_ulat, validate_ulon)

In [ ]:
validation_points=pd.DataFrame(val_points)
len(validation_points)

In [ ]:
validation_points = validation_points.sample(frac=1).reset_index(drop=True)
validation_points.head()

In [ ]:
len(validation_points)

In [ ]:
validation_venues = getNearbyVenues(latitudes = validation_points[0],
                              longitudes =  validation_points[1]
                               )

Generate completely new data to try 

In [ ]:
len(validation_venues['Point Name'].unique())

In [ ]:
validation_venues.tail()

In [ ]:
validation_venues.head()

In [ ]:
#nelat = 41.962643
#nelng = -87.757414
#swlat = 41.903702
#swlng = -87.896807
nelat = validate_llat
nelng = validate_llon
swlat = validate_ulat
swlng = validate_ulon
sb_locations = []
#for lat, lng in zip(validation_answer_key['point_lat'],validation_answer_key['point_lon']):   
count=0
for lat, lng in zip(validation_points[0],validation_points[1]):    
    sb_location_res = getStarbucksLocations(lat, lng, limit=200)
    for sb_location in sb_location_res:
        print("lat=" + str(sb_location[0]) + "lon=" + str(sb_location[1])) 
        sb_locations.append((sb_location[0],sb_location[1]))   
        #print(sb_location)
    if(count%20 == 0):
                print("\r{0}".format(round((float(count)/len(validation_points[0]))*100),2)+"% done")
    count = count + 1

In [ ]:
validation_venues.tail()

In [ ]:
sb_locations_df = pd.DataFrame(sb_locations)
sb_locations_df.tail()

In [ ]:
validation_venues['Venue'].value_counts()

In [ ]:
# one hot encoding
validation_onehot = pd.get_dummies(validation_venues[['Venue Category']], prefix="", prefix_sep="")

In [ ]:
validation_onehot.head()

In [ ]:
# add neighborhood column back to dataframe
validation_onehot['point_name'] = validation_venues['Point Name'] 
validation_onehot['point_lat'] = validation_venues['Neighborhood Latitude'] 
validation_onehot['point_lon'] = validation_venues['Neighborhood Longitude'] 
validation_onehot['isStarbucks'] = (validation_venues['Venue'] == "Starbucks")# should do a better fuzzy match
# move neighborhood column to the first column
#validation_fixed_columns = [validation_onehot.columns[-3]] + [validation_onehot.columns[-2]] + [validation_onehot.columns[-1]] + list(validation_onehot.columns[:-3])

In [ ]:
# move neighborhood column to the first column
validation_fixed_columns = [validation_onehot.columns[-3]] + [validation_onehot.columns[-2]] + [validation_onehot.columns[-1]] + list(validation_onehot.columns[:-3])

In [ ]:
len(validation_fixed_columns)

In [ ]:
validation_onehot.columns

In [ ]:
validation_fixed_columns = [validation_onehot.columns[-3]] + [validation_onehot.columns[-2]] + [validation_onehot.columns[-1]] + list(validation_onehot.columns[:-3])

In [ ]:
validation_onehot = validation_onehot[validation_fixed_columns]

In [ ]:
validation_onehot.isStarbucks = validation_onehot.isStarbucks.astype(int)
validation_onehot.head()
validation_onehot['isStarbucks'].value_counts()
validation_venues.head()
len(validation_onehot)
validation_onehot.head()
validation_onehot.shape
validation_answer_key = validation_onehot
validation_answer_key.head()

In [ ]:
test_answer_key = onehot
test_answer_key.head()

In [ ]:
validation_answer_key = validation_answer_key.groupby('point_name').mean()
validation_answer_key.head()
validation_answer_key.shape
validation_answer_key = validation_onehot.groupby('point_name').mean().reset_index()
validation_answer_key.head()

In [ ]:
validation_points_near_starbucks = validation_onehot[validation_onehot.isStarbucks > 0]["point_name"]
if eliminate_starbucks_venue_rows == True:
    #keep the isStarbucks flag but throw out other venue info
    ##points_near_starbucks = onehot[onehot.isStarbucks > 0]["point_name"]
    validation_indexes_to_remember = validation_onehot[validation_onehot.isStarbucks > 0].index
    validation_points_near_starbucks
validation_points_near_starbucks_df = pd.DataFrame(validation_points_near_starbucks)
validation_points_near_starbucks_df.head()
for point_name in validation_points_near_starbucks_df.point_name:
    validation_onehot.loc[validation_onehot.point_name == point_name,"isStarbucks"] = 1
len(validation_onehot)

In [ ]:
 if eliminate_starbucks_venue_rows == True:
    for ind in validation_indexes_to_remember:
        validation_onehot = validation_onehot.drop([ind], axis=0)
len(validation_onehot)

len(validation_onehot.columns)
validation_onehot['number_of_venues_within_radius'] = 1
len(validation_onehot.columns)
validation_onehot['point_name']       

In [ ]:
# create the new feature 'number_of_venues_within_radius'
# have to combine means and counts
validation_grouper = validation_onehot.groupby('point_name').mean().reset_index()
validation_counter = validation_onehot.groupby('point_name').count().reset_index()
validation_grouped = validation_grouper
validation_grouped.iloc[:,-1] = validation_counter.iloc[:,-1]
validation_max_of_venue_count = validation_grouped.iloc[:,-1].max()
# normalize the new feature
validation_grouped.iloc[:,-1] = validation_grouped.iloc[:,-1]/validation_max_of_venue_count
validation_grouped.iloc[:,-1].describe()

In [ ]:
verbose = False
num_top_venues = 5
count = 0
for point in validation_grouped['point_name']:
    validation_temp = validation_grouped[validation_grouped['point_name'] == point].T.reset_index()
    validation_temp.columns = ['venue','freq']
    validation_temp = validation_temp.iloc[2:]
    validation_temp['freq'] = validation_temp['freq'].astype(float)
    validation_temp = validation_temp.round({'freq': 2})
    if verbose == True:
        print(str(count)+"----"+str(point)+"----")
        print(validation_temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
        print('\n')
    else:
        if count % 100 == 0:
            print("----"+point+"----")
            print(validation_temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
            print('\n')
    count = count + 1


In [ ]:
validation_grouped.shape

In [ ]:
validation_grouped.head()
validation_y = validation_grouped['isStarbucks'] > 0
validation_y = pd.DataFrame(validation_y)
validation_y = validation_y.isStarbucks.astype(int)
print(validation_y.value_counts())
validation_X = validation_grouped
print(validation_X.shape)
validation_map_data = validation_X
validation_X.drop(columns=['point_name', 'point_lat', 'point_lon', 'isStarbucks'], inplace=True)
validation_X.columns

In [ ]:
validation_X['random'] = np.random.random(size=len(validation_X))
print(validation_X.columns)
print(validation_X.shape)
validation_pdy=pd.DataFrame(validation_y)
validation_pdy['isStarbucks'].value_counts()

In [ ]:
#yikes - my features don't match
X.head()

In [ ]:
X.shape

In [ ]:
#Let's see if we can fit this square peg into that round hole
matchup_validation_X = []
matchup_validation_X = pd.DataFrame(matchup_validation_X)

In [ ]:
matchup_validation_X = pd.DataFrame(0, index=np.arange(len(validation_X)), columns=X.columns)

In [ ]:
for val_col in validation_X.columns:
    matchup_validation_X[val_col] = validation_X[val_col]

In [ ]:
matchup_validation_X.shape

Looks like we have some extra columns at the end
Would be nice to have incorporated those extra rows at the beginning
For now, we just throw those extra columns out

In [ ]:
while len(matchup_validation_X.columns) > len(X.columns):
    matchup_validation_X.drop(matchup_validation_X.columns[len(matchup_validation_X.columns)-1], axis=1, inplace=True)


In [ ]:
matchup_validation_X.shape

In [ ]:
pd.DataFrame(X).shape

In [ ]:
validation_X = matchup_validation_X

In [ ]:
X_val=validation_X
y_val=validation_y

In [ ]:
###############
##############
#### validate end ###
################

In [ ]:
corr = X.corr()
sns.heatmap(corr)

Features are predominantly uncorrelated

In [ ]:
c = X.corr().abs()
s = c.unstack()
so = s.sort_values(kind="quicksort")

In [ ]:
#c.head()

In [ ]:
strong_correlations = []
correlation_threshold = 0.7
for colname in c.columns:
    if verbose == True:
        print("check for features strongly coorelated with " + colname)
    for i in range(0, len(c)):
        this_correlation = c[colname][i]
        if this_correlation >= correlation_threshold:
            if colname != c.index[i]: #don't act on self correlations
                if verbose == True:
                    print("   " + c.index[i] + " - " + str(this_correlation))
                strong_correlations.append((colname,c.index[i],this_correlation))

Whether or not I do anything about correlated features, they are interesting to look at.<br />
Some of them appear to be negatively correlated even I only show the magnitude of the correlation here.<br />
Some of the correlations look like they are due to small samples, such as the Roller Rink in San Francisco that is perfectly correlated with a Spiritual Center. I've actually skated there.<br />
Some correlations look meaningful, such as the correlation between aquariums and zoos.


In [ ]:
strong_correlations_df = pd.DataFrame(strong_correlations)

In [ ]:
strong_correlations_df["feature_pair"]=strong_correlations_df[0]+"-"+strong_correlations_df[1]

In [ ]:
strong_correlations_df.drop([0,1],axis=1, inplace=True)

In [ ]:
strong_correlations_df.rename(columns= {2: "correlaton"}, inplace=True)
strong_correlations_df.sort_values(by="correlaton",ascending=False,inplace=True)
strong_correlations_df.set_index("feature_pair", drop=True)

I wanted to see if there were strong correlations in the features, so I could remove some.<br />
With 400 or so features and something like 600 data points, I thought it would be good to remove redundant features.<br />
I found strongly correlated features, but my performance was good.<br /> I learned that Random Forests can be
tolerant of correlated features, so removing them seems uneccesary.

In [ ]:
#strong_correlations_df

In [ ]:
RF_all_features_1 = RF("RF_all_features_1", X_train, y_train, X_test, y_test, X_val, y_val)
RF_all_features_1.fit()
RF_all_features_1.get_and_print_results()
RF_all_features_1.feature_importance_df

If the test accuracy is a lot lower than the train accuracy, it could be a sign of overfitting

ROC Area Under Curve is a good way of measuring how well the binary classifier separates the two classes, even if the classes are unbalanced

In [ ]:
RF_all_features_1.rfc

In [ ]:
estimator =  RF_all_features_1.get_estimator(0)

In [ ]:
estimator

In [ ]:
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = X.columns,
                class_names = ('near-Starbucks','not-near'),
                rounded = True, proportion = True, 
                precision = 2, filled = True)

Display a tree<br />
Note: this is only one tree <br />
Random Forest takes a vote from many trees <br />
It is possible however, to use only one tree (with some loss of accuracy),<br />
and interpret how the classifier is working

In [ ]:
# Convert to png using system command (requires Graphviz)
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
Image(filename = 'tree.png')

Can we do better using recursive feature elimination?

In [ ]:
rf_limited_features = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=0)  
rfecv = RFECV(estimator=rf_limited_features, step=1, cv=2, scoring='roc_auc', verbose=2) 
selector=rfecv.fit(X_train, y_train)

In [ ]:
#sk.metrics.SCORERS.keys()#this shows the scoring metric choices

In [ ]:
reduced_features_preds = selector.predict(X_train)

In [ ]:
selector.score(X_train,y_train)

In [ ]:
selector_score = selector.score(X_test,y_test)
selector_score

In [ ]:
selector.n_features_

In [ ]:
selector

In [ ]:
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

Selector picked 140 features, but that looks like a noisy spike<br />
Half that (~70) looks like it would do fine, but there is no real loss by keeping extra features<br />
Random Forest model does fairly well with extra features

Give svm a try

In [ ]:
SVM_rbf = SVM("SVM (with rbf kernel)",'rbf', X_train, y_train, X_test, y_test, X_val, y_val)
SVM_rbf.fit()
SVM_rbf.predict(X_test)
SVM_rbf.print_scores()

In [ ]:
SVM_linear = SVM("SVM (with linear kernel)",'linear', X_train, y_train, X_test, y_test, X_val, y_val)
SVM_linear.fit()
SVM_linear.predict(X_test)
SVM_linear.print_scores()

In [ ]:
SVM_poly = SVM("SVM (with poly kernel)",'poly', X_train, y_train, X_test, y_test, X_val, y_val)
SVM_poly.fit()
SVM_poly.predict(X_test)
SVM_poly.print_scores()

In [ ]:
SVM_sigmoid = SVM("SVM (with sigmoid kernel)",'sigmoid', X_train, y_train, X_test, y_test, X_val, y_val)
SVM_sigmoid.fit()
SVM_sigmoid.predict(X_test)
SVM_sigmoid.print_scores()

Looks like RandomForest and SVM with an rbf kernel do well<br />
Maybe we should try those with smaller feature sets

In [ ]:
#tried dropping my added feature but lets keep it
#X_train.drop(['number_of_venues_within_radius'], axis=1)
#X_test.drop(['number_of_venues_within_radius'], axis=1)

In [ ]:
#RF_2 = RF("Random Forest #2", X_train, y_train, X_test, y_test, X_val, y_val)
#RF_2.fit()
#RF_2.get_and_print_results()
#RF_2.feature_importance_df



In [ ]:
map_testing_data = folium.Map(location=[ulat, ulon], zoom_start=12)

In [ ]:
test_answer_key_unique = pd.DataFrame(test_answer_key.groupby(['point_name']))

In [ ]:
test_answer_key_unique = test_answer_key[['point_name','point_lat','point_lon','isStarbucks']]

In [ ]:
last_point_name = ''
# add markers for my predicted classes to map
#for lat, lng in zip(validation_venues['Neighborhood Latitude'],validation_venues['Neighborhood Longitude']):
##for lat, lng in zip(validation_points[0],validation_points[1]):
count = 0
for point_name, lat, lng, isSB in zip(test_answer_key['point_name'],test_answer_key['point_lat'],test_answer_key['point_lon'],test_answer_key['isStarbucks']):    

    
    if point_name != last_point_name:
        
        if isSB > 0 :
            color_str = 'green'
            #print(color_str)
        else:
            color_str = 'blue'
            #print(color_str)
  
        if count < 2000:
            folium.CircleMarker(
                [lat, lng],
                radius=5,
                color=color_str,        
                fill=False,
                fill_color='#3186cc',
                fill_opacity=0.7,
                parse_html=False).add_to(map_testing_data)  
        count = count + 1
    #print(point_name)     
        sb_test_locs = getStarbucksLocations(lat, lng, limit=200)
        sb_test_locs_df = pd.DataFrame(sb_test_locs)
        sb_test_locs_df.head()

        if len(sb_test_locs) > 0: 
            for sblat, sblng in zip(sb_test_locs_df[0],sb_test_locs_df[1]):                  
                folium.CircleMarker(
                    [sblat, sblng],
                    radius=5,
                    color='red',        
                    fill=False,
                    fill_opacity=0.7,
                    parse_html=False).add_to(map_testing_data)  
        count = count + 1
    last_point_name = point_name

In [ ]:
map_testing_data_legend_html =   '''
                <div style="position: fixed; 
                            bottom: 50px; left: 500px; width: 168px; height: 130px; 
                            border:2px solid grey; z-index:9999; font-size:14px;
                            #background: white;
                            ">
                              <div style="font-size:10px;"><br />&nbsp; Class 1 is predicted to be within <br /> 
                              &nbsp; 300m of a Starbucks location <br />
                              &nbsp;<br />
                             
                              </div>
                         
                              &nbsp;&nbsp;&nbsp; predicted class 0: &nbsp; <i class="fa fa-circle-o style="color:blue"></i><br>
                              &nbsp;&nbsp;&nbsp; predicted class 1: &nbsp; <i class="fa fa-circle-o style="color:green"></i><br>                              
                              &nbsp;&nbsp;&nbsp; true class 1: &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <i class="fa fa-circle-o" style="color:red">
                              </i>
                </div>
                ''' 
map_testing_data.get_root().html.add_child(folium.Element(map_testing_data_legend_html))   

In [ ]:
map_testing_data

In [ ]:
validate_center_lat = validate_llat+(validate_ulat-validate_llat)/2
validate_center_lon = validate_llon+(validate_ulon-validate_llon)/2
validation_map = folium.Map(location=[validate_center_lat, validate_center_lon], zoom_start=12)

In [ ]:
# add markers for known Starbucks to map
for lat, lng in zip(sb_locations_df[0],sb_locations_df[1]):    
    color_str = 'red'

    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=color_str,        
        fill=False,
        #fill_color='#3186cc',
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(validation_map)  

In [ ]:

map_validation_data_legend_html =   '''
                <div style="position: fixed; 
                            bottom: 50px; left: 30px; width: 168px; height: 130px; 
                            border:2px solid grey; z-index:9999; font-size:14px;
                            #background: white;
                            ">
                              <div style="font-size:10px;"><br />&nbsp; Class 1 is predicted<br /> 
                              &nbsp; to be within 300m of <br />
                              &nbsp; aStarbucks location<br />
                             
                              </div>
                              
                              &nbsp;&nbsp;&nbsp; predicted class 0: &nbsp; <i class="fa fa-circle-o style="color:blue"></i><br>
                              &nbsp;&nbsp;&nbsp; predicted class 1: &nbsp; <i class="fa fa-circle-o style="color:green"></i><br>                              
                              &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; true class 1: &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <i class="fa fa-circle-o" style="color:red">
                              </i>
                </div>
                ''' 
validation_map.get_root().html.add_child(folium.Element(map_validation_data_legend_html))

In [ ]:
# add markers for my predicted classes to map
#for lat, lng in zip(validation_venues['Neighborhood Latitude'],validation_venues['Neighborhood Longitude']):
##for lat, lng in zip(validation_points[0],validation_points[1]):
for lat, lng, isSB in zip(validation_answer_key['point_lat'],validation_answer_key['point_lon'],validation_answer_key['isStarbucks']):    

    if isSB > 0 :
        color_str = 'green'
    else:
        color_str = 'blue'

    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=color_str,        
        fill=False,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(validation_map)  
    


In [ ]:
validation_map

In [ ]:
##### end validate #####

In [ ]:
#feature_importance_df = RF_2.feature_importance_df
feature_importance_df = RF_all_features_1.feature_importance_df


In [ ]:
#Save original matrices before you start cutting them up (just for safe keeping in case needed again)
X_train_orig = X_train.copy(deep=True)
y_train_orig = y_train.copy(deep=True)
X_test_orig = X_test.copy(deep=True)
y_test_orig = y_test.copy(deep=True)
X_val_orig = X_val.copy(deep=True)
y_val_orig = y_val.copy(deep=True)
feature_importance_df_orig = feature_importance_df.copy(deep=True)

In [ ]:
verbose = False

def drop_features_below_cutoff(cutoff, feature_importance_df, this_df):
    count = 0
    num_to_do = feature_importance_df.size - cutoff
    for feature in feature_importance_df[cutoff:feature_importance_df.size].index: 
        if verbose == True:
            print("removing feature: " + str(feature) + " with importance = " + str(feature_importance_df.loc[feature]['Feature Importance']))
        try:
            this_df.drop(columns=[feature], inplace=True)
        except:
            if verbose == True:
                print("EXCEPTION - removing feature: " + str(feature) + " with importance = " + str(feature_importance_df.loc[feature]['Feature Importance']))
        
        
        # progress indicator for the impatient among us
        if(count%20 == 0):
            pass
           # print("\r{0}".format(round((float(count)/num_to_do*100),2)+"% done"))
           # print("\r{0}".format(round((float(count/num_to_do*100)),2)+"% done"))            
            
        count = count + 1            
    return(this_df)

In [ ]:
verbose = False
def drop_features_on_all_sets(cutoff, feature_importance_df, X_train, X_test, X_val):
    X_train = drop_features_below_cutoff(cutoff, feature_importance_df, X_train)
    X_test = drop_features_below_cutoff(cutoff, feature_importance_df, X_test)
    X_val = drop_features_below_cutoff(cutoff, feature_importance_df, X_val) 
    return(X_train, X_test, X_val)

In [ ]:
len(X_train.columns)
X_train = X_train_orig

In [ ]:
len(X_train.columns)


In [ ]:
#X_train = X_train_orig
#X_test = X_test_orig
#X_val = X_val_orig

In [ ]:
X_val_orig.columns

In [ ]:
feature_importance_df.iloc[112].name


In [ ]:
X_train.columns

In [ ]:
# drop features with importance below cutoff
cutoff = selector.n_features_
X_train, X_test, X_val  = drop_features_on_all_sets(cutoff, feature_importance_df, X_train, X_test, X_val)

In [ ]:

X_train = X_train_orig
print(X_train_orig.columns)
print(X_test_orig.columns)
print(X_val_orig.columns)

X_train.columns[100:150]

In [ ]:
X_test.columns

In [ ]:
X_train.columns

In [ ]:
X_val.columns

In [ ]:
len(X_train.columns)

In [ ]:
X_train.columns

In [ ]:
X_test.describe()

In [ ]:
X_val.describe()

In [ ]:
RF_3 = RF("Random Forest #3", X_train, y_train, X_test, y_test, X_val, y_val)
RF_3.fit()
RF_3.get_and_print_results()
RF_3.feature_importance_df

In [ ]:
cutoff = 50
# drop features with importance below cutoff
X_train, X_test, X_val  = drop_features_on_all_sets(cutoff, feature_importance_df, X_train, X_test, X_val)

#X_train = drop_features_below_cutoff(cutoff, feature_importance_df, X_train)
#X_test = drop_features_below_cutoff(cutoff, feature_importance_df, X_test)
#X_val = drop_features_below_cutoff(cutoff, feature_importance_df, X_val)


In [ ]:
len(X_train.columns)

In [ ]:
len(X_test.columns)

In [ ]:
len(X_val.columns)

In [ ]:
RF_4 = RF("Random Forest #4", X_train, y_train, X_test, y_test, X_val, y_val)
RF_4.fit()
RF_4.get_and_print_results()
RF_4.feature_importance_df


Now let's set the cutoff to the feature importance level where the selector plot first hits<br />
the high plateau

In [ ]:
cutoff = 20
# drop features with importance below cutoff
X_train, X_test, X_val  = drop_features_on_all_sets(cutoff, feature_importance_df, X_train, X_test, X_val)

In [ ]:
print(X_train_orig.columns)
print(X_test_orig.columns)
print(X_val_orig.columns)

In [ ]:
# drop features with importance below cutoff
#cutoff = 20
#X_train = drop_features_below_cutoff(cutoff, feature_importance_df, X_train)
#X_test = drop_features_below_cutoff(cutoff, feature_importance_df, X_test)   
#X_val = drop_features_below_cutoff(cutoff, feature_importance_df, X_val)   

In [ ]:
RF_5 = RF("Random Forest #5", X_train, y_train, X_test, y_test, X_val, y_val)
RF_5.fit()
RF_5.get_and_print_results()
RF_5.feature_importance_df

In [ ]:
feature_importance_df

In [ ]:
# Get the position of the random feature in the sorted feature_importances_df
random_position = len(feature_importance_df) # default to max
for i in range(0, len(feature_importance_df)):
    #print(feature_importance_df.iloc[i].name)
    if feature_importance_df.iloc[i].name == 'random':
        random_position = i
        break

In [ ]:
random_position


Now let's set the cutoff to the feature importance level of the random feature

In [ ]:
# drop features with importance below cutoff
cutoff = random_position
X_train = drop_features_below_cutoff(cutoff, feature_importance_df, X_train)
X_test = drop_features_below_cutoff(cutoff, feature_importance_df, X_test)    
X_val = drop_features_below_cutoff(cutoff, feature_importance_df, X_val) 

In [ ]:
RF_6 = RF("Random Forest #6", X_train, y_train, X_test, y_test, X_val, y_val)
RF_6.fit()
RF_6.get_and_print_results()
RF_6.feature_importance_df